In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

In [61]:
# Exportando bases

#bio_atleta = pd.read_excel("Olympic_Athlete_Bio.xlsx")
#medalhas = pd.read_excel("Olympic_Games_Medal_Tally.xlsx")
#resultado_atleta = pd.read_excel("Olympic_Athlete_Event_Results.xlsx")
#pais_edicao = pd.read_excel("Olympics_Games.xlsx")
medalhas = pd.read_excel("Olympic_Games_Medal_Tally1.xlsx")

In [ ]:
display(resultado_atleta)

,edition,edition_id,country_noc,sport,event,result_id,athlete,athlete_id,pos,medal,isTeamSport
0,1908 Summer Olympics,5,ANZ,Athletics,100 metres Men,56265,Ernest Hutcheon,64710,DNS,na,False
1,1908 Summer Olympics,5,ANZ,Athletics,400 metres Men,56313,Henry Murray,64756,DNS,na,False
2,1908 Summer Olympics,5,ANZ,Athletics,800 metres Men,56338,Harvey Sutton,64808,3 h8 r1/2,na,False
3,1908 Summer Olympics,5,ANZ,Athletics,800 metres Men,56338,Guy Haskins,922519,DNS,na,False
4,1908 Summer Olympics,5,ANZ,Athletics,800 metres Men,56338,Joseph Lynch,64735,DNS,na,False
...,...,...,...,...,...,...,...,...,...,...,...
253177,2020 Summer Olympics,61,USA,Wrestling,Featherweight Freestyle Women,19009677,Jacarra Winchester,147307,5,na,False
253178,2020 Summer Olympics,61,USA,Wrestling,Lightweight Freestyle Women,19009706,Helen Maroulis,132194,3,Bronze,False
253179,2020 Summer Olympics,61,USA,Wrestling,Middleweight Freestyle Women,19009735,Kayla Miracle,147308,12,na,False
253180,2020 Summer Olympics,61,USA,Wrestling,Light-Heavyweight Freestyle Women,19009764,Tamyra Mensah-Stock,147310,1,Gold,False


### Análise Evolução participação de gênero

In [ ]:
display(bio_atleta)

,athlete_id,name,sex
0,16809,Károly Teppert,Male
1,43737,Andrzej Socharski,Male
2,50147,Nathalie Wunderlich,Female
3,5085,Miha Lokar,Male
4,136329,Austin Hack,Male
...,...,...,...
154089,25136,Jean Palluch,Male
154090,701697,Hendrik Mangelaar Meertens,Male
154091,11757,Jo Hyeong-Won,Male
154092,21666,Raúl Maroto,Male


In [31]:
#Contando o número de participante por ano e por gênero

# Criando a coluna year
resultado_atleta['year'] = resultado_atleta['edition'].str.extract(r'(\d{4})').astype(int)

# Unindo o df resultado_atleta e bio_atleta por athlete_id

atleta_genero = bio_atleta[['athlete_id','sex']]
resultado_atleta_bio = resultado_atleta.merge(atleta_genero, on='athlete_id')

#Removendo id de atletas duplicados
resultado_atleta_bio = resultado_atleta_bio.drop_duplicates(subset='athlete_id')

# Agrupando os gêneros pelos anos das edições
genero_ano = resultado_atleta_bio.groupby(['year','sex'])['athlete_id'].count().reset_index()
genero_ano = genero_ano.rename(columns = {'athlete_id':'count'})

# Acrescentando novas linhas
new_rows = pd.DataFrame([
    {'year': 2024, 'sex': 'Male', 'count': 5250},
    {'year': 2024, 'sex': 'Female', 'count': 5250}
])

# Adicionar novas linhas ao genero_ano
genero_ano = pd.concat([genero_ano, new_rows], ignore_index=True)

In [32]:
# Visualização
fig = go.Figure()

#Traduzindo
nomes_legenda = {'Male': 'Masculino', 'Female': 'Feminino'}

# Adicionando uma linha para cada valor único em 'sex'
for sexo in genero_ano['sex'].unique():
    df_sexo = genero_ano[genero_ano['sex'] == sexo]
    fig.add_trace(go.Scatter(x=df_sexo['year'], y=df_sexo['count'],
                             mode='lines+markers', name=nomes_legenda[sexo]))

# Layout do gráfico com fundo branco e título centralizado
fig.update_layout(
    title={
        'text': 'Representatividade por gênero nos Jogos Olímpicos',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='Edições',
    yaxis_title='Número de atletas',
    plot_bgcolor='white',  # Fundo da área do gráfico
    paper_bgcolor='white',  # Fundo do papel do gráfico
    yaxis=dict(
        tickformat='.'  # Números inteiros
    ),
    shapes=[
        # Linha vertical destacando o ano de 2024
        dict(
            type="line",
            xref="x",
            yref="paper",
            x0=2024,
            y0=0,
            x1=2024,
            y1=1,
            line=dict(
                color="Red",
                width=2,
                dash="dashdot",
            ),
        )
    ]
)

fig.show()


### Número medalhas Brasil

In [7]:
# Filtrando o df medalhas para o país igual brasil
medalhas_brasil = medalhas[medalhas['country'] == 'Brazil']

# Somando as medalhas
ouro = medalhas_brasil['gold'].sum()
prata = medalhas_brasil['silver'].sum()
bronze = medalhas_brasil['bronze'].sum()

# Criando o df medalhas_bra
medalhas_bra = pd.DataFrame({
    'Tipo': ['Ouro', 'Prata','Bronze'],
    'Medalhas': [ouro,prata,bronze],
})


In [8]:
# Visualização
fig = go.Figure(data=[go.Pie(
    labels=medalhas_bra['Tipo'],
    values=medalhas_bra['Medalhas'],
    textinfo='value',  # Mostra rótulos e percentuais
    hole=0.3,  # Cria um gráfico de pizza com um buraco no meio (para estilo de rosquinha)
    marker=dict(colors=['gold', 'silver','brown'])  # Define cores para as fatias
)])

fig.update_layout(
    title={
        'text': 'Medalhas conquistadas pelo Brasil ',
        'x': 0.5,
        'xanchor': 'center'
    },
    paper_bgcolor='white',
    plot_bgcolor='white'
)

fig.show()

### Medalhas conquistadas por outros países

In [ ]:
medalhas

,edition,edition_id,year,country,country_noc,gold,silver,bronze,total
0,1896 Summer Olympics,1.0,1896.0,United States,USA,11.0,7.0,2.0,20.0
1,1896 Summer Olympics,1.0,1896.0,Greece,GRE,10.0,18.0,19.0,47.0
2,1896 Summer Olympics,1.0,1896.0,Germany,GER,6.0,5.0,2.0,13.0
3,1896 Summer Olympics,1.0,1896.0,France,FRA,5.0,4.0,2.0,11.0
4,1896 Summer Olympics,1.0,1896.0,Great Britain,GBR,2.0,3.0,2.0,7.0
...,...,...,...,...,...,...,...,...,...
1802,1906 Intercalated,4.0,1906.0,Canada,CAN,1.0,1.0,0.0,2.0
1803,1906 Intercalated,4.0,1906.0,Norway,NOR,1.0,1.0,0.0,2.0
1804,1906 Intercalated,4.0,1906.0,Netherlands,NED,0.0,1.0,2.0,3.0
1805,1906 Intercalated,4.0,1906.0,Australia,AUS,0.0,0.0,3.0,3.0


In [72]:
olimpiadas_verao = medalhas[medalhas.apply(lambda x: x.astype(str).str.contains('summer', case=False).any(), axis=1)]

# Agrupando o total de medalhas por países (top 10)
olimpiadas_verao = olimpiadas_verao.groupby('country')['total'].sum().reset_index()
olimpiadas_verao = olimpiadas_verao.sort_values(by='total', ascending=False).head(10)
olimpiadas_verao = olimpiadas_verao.sort_values(by='total', ascending=True).head(10)

nomes_legenda = {'United States': 'Estados Unidos', 'Soviet Union': 'Rússia','Great Britain':'Grã-Bretanha','France':'França','Germany':'Alemanha','Italy':'Itália',
                 'Australia':'Austrália','Hungary':'Hungria','Sweden':'Suécia',"People's Republic of China":"China"}

# Traduzindo o nome dos países
olimpiadas_verao['country'] = olimpiadas_verao['country'].map(nomes_legenda)

In [73]:
olimpiadas_verao

,country,total
124,Suécia,504
55,Hungria,515
5,Austrália,541
64,Itália,629
98,China,634
43,França,772
46,Alemanha,791
48,Grã-Bretanha,928
119,Rússia,1010
142,Estados Unidos,2655


In [95]:
# Gráfico de barras deitado
fig = go.Figure(data=[go.Bar(
    y=olimpiadas_verao['country'],   # Eixo Y é categórico
    x=olimpiadas_verao['total'], # Eixo X é contínuo
    orientation='h',         # Configura o gráfico para ser horizontal
    marker=dict(color=['gold','LightGreen','lightblue','red','limegreen','darkturquoise','gray','deepskyblue','salmon','mediumblue']
     )  # Cor das barras
)])

# Configurar o layout do gráfico
fig.update_layout(
    title={
        'text': 'Os 10 países com mais medalhas acumuladas - Olimpíadas de Verão ',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='Medalhas',
    yaxis_title='Países',
    paper_bgcolor='white',  # Fundo do papel do gráfico
    plot_bgcolor='white'   # Fundo da área do gráfico
)

# Mostrar o gráfico
fig.show()

In [87]:
olimpiadas_inverno = medalhas[medalhas.apply(lambda x: x.astype(str).str.contains('winter', case=False).any(), axis=1)]

# Agrupando o total de medalhas por países (top 10)
olimpiadas_inverno = olimpiadas_inverno.groupby('country')['total'].sum().reset_index()
olimpiadas_inverno = olimpiadas_inverno.sort_values(by='total', ascending=False).head(10)
olimpiadas_inverno = olimpiadas_inverno.sort_values(by='total', ascending=True).head(10)

nomes_legenda = {'Norway': 'Noruega', 'Soviet Union': 'Rússia','United States':'Estados Unidos','Finland':'Finlândia','Germany':'Alemanha','Sweden':'Suécia',
                 'Switzerland':'Suíça','Netherlands':'Países Baixos','Sweden':'Suécia',"Canada":"Canadá","Austria":"Áustria"}

#Traduzindo o nome dos países
olimpiadas_inverno['country'] = olimpiadas_inverno['country'].map(nomes_legenda)


In [94]:
# Gráfico de barras deitado
fig = go.Figure(data=[go.Bar(
    y=olimpiadas_inverno['country'],   # Eixo Y é categórico
    x=olimpiadas_inverno['total'], # Eixo X é contínuo
    orientation='h',         # Configura o gráfico para ser horizontal
    marker=dict(color=['orange','gold','lightblue','red','salmon','darkturquoise','gray','deepskyblue','blue','maroon']
     )  # Cor das barras
)])

# Configurar o layout do gráfico
fig.update_layout(
    title={
        'text': 'Os 10 países com mais medalhas acumuladas - Olimpíadas de Inverno',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='Medalhas',
    yaxis_title='Países',
    paper_bgcolor='white',  # Fundo do papel do gráfico
    plot_bgcolor='white'   # Fundo da área do gráfico
)

# Mostrar o gráfico
fig.show()

### Países que já sediaram as Olimpíadas



In [96]:
pais_edicao['country_noc'].unique()

array(['GRE', 'FRA', 'USA', 'GBR', 'SWE', 'GER', 'BEL', 'NED', 'FIN',
       'AUS', 'ITA', 'JPN', 'MEX', 'CAN', 'URS', 'KOR', 'ESP', 'CHN',
       'BRA'], dtype=object)

In [13]:
# Trocando os códigos para nomes de países
country_mapping = {'GRE': 'Greece','FRA': 'France','USA': 'United States','GBR': 'Great Britain','SWE': 'Sweden','GER': 'Germany','BEL': 'Belgium','NED': 'Netherlands','FIN': 'Finland',
                   'AUS': 'Australia','ITA': 'Italy','JPN': 'Japan','MEX': 'Mexico','CAN': 'Canada','URS': 'Soviet Union (now Russia)','KOR': 'South Korea','ESP': 'Spain','CHN': 'China','BRA': 'Brazil'
}

pais_edicao['country'] = pais_edicao['country_noc'].map(country_mapping)

# Contar a ocorrência de cada país
country_counts = pais_edicao['country'].value_counts().reset_index()

# Renomear as colunas
country_counts.columns = ['country', 'count']


In [15]:
# Mapear contagens para cores específicas
color_discrete_map = {
    '1': '#ADD8E6',  # Cor para contagem 1
    '2': '#40E0D0',  # Cor para contagem 2
    '3': '#4682B4',  # Cor para contagem 3
    '4': '#00008B'   # Cor para contagem 4
}


# Criar uma coluna categórica para as contagens
country_counts['count_str'] = country_counts['count'].astype(str)

# Criar um mapa coroplético
fig = px.choropleth(country_counts,
                    locations='country',
                    locationmode='country names',
                    color='count_str',
                    hover_name='country',
                    title='Países Anfitriões dos Jogos Olímpicos',
                    color_discrete_map=color_discrete_map,
                    category_orders={'count_str': ['1', '2', '3', '4']})

# Atualizar o layout para incluir a legenda de blocos de cores
fig.update_layout(
    legend_title_text='Ocorrências',
    legend=dict(
        itemsizing='constant',
        font=dict(size=12)
    )
)


# Mostrar o gráfico
fig.show()


### Modalidades mais vencedoras

In [33]:
resultado_atleta_brasil = resultado_atleta[resultado_atleta['country_noc'] == 'BRA']
resultado_atleta_brasil = resultado_atleta_brasil.replace('na',0)
resultado_atleta_brasil = resultado_atleta_brasil[resultado_atleta_brasil['medal'] != 0]

medal = {'Gold':1,'Silver':1,'Bronze':1}

resultado_atleta_brasil['medal_count'] = resultado_atleta_brasil['medal'].map(medal)

# Remover linhas duplicadas com base em result_id
resultado_atleta_brasil = resultado_atleta_brasil.drop_duplicates('result_id')

sports_translation = {
    'Judo': 'Judô',
    'Sailing': 'Vela',
    'Athletics': 'Atletismo',
    'Swimming': 'Natação',
    'Volleyball': 'Voleibol',
    'Beach Volleyball': 'Vôlei de Praia',
    'Football': 'Futebol',
    'Boxing': 'Boxe',
    'Artistic Gymnastics': 'Ginástica Artística',
    'Basketball': 'Basquete',
    'Canoe Sprint': 'Canoagem de Velocidade',
    'Shooting': 'Tiro Esportivo',
    'Equestrian Jumping': 'Hipismo de Saltos',
    'Skateboarding': 'Skate',
    'Taekwondo': 'Taekwondo',
    'Marathon Swimming': 'Maratona Aquática',
    'Modern Pentathlon': 'Pentatlo Moderno',
    'Surfing': 'Surfe',
    'Tennis': 'Tênis'
}

#Contando o número de medalhas por modalidades
modalidades = resultado_atleta_brasil.groupby('sport')['medal_count'].sum().sort_values(ascending=False).reset_index()

modalidades['sport'] = modalidades['sport'].map(sports_translation)

In [34]:
modalidades

,sport,medal_count
0,Judô,24
1,Vela,19
2,Atletismo,19
3,Natação,15
4,Voleibol,11
5,Vôlei de Praia,10
6,Futebol,9
7,Boxe,8
8,Ginástica Artística,5
9,Basquete,5


In [60]:
# Criar o gráfico de barras
fig = px.bar(modalidades,
             x='sport',
             y='medal_count',
             text='medal_count',
             title='Número de medalhas conquistadas por modalidades',
             labels={'sport': 'Esporte', 'medal_count': 'Contagem de Medalhas'},
            )

# Configurar o layout do gráfico
fig.update_layout(
    title={
        'text': 'Número de medalhas conquistadas por modalidades',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='',
    yaxis_title='Medalhas',
        yaxis=dict(
        showticklabels=False,
        ),  # Remove os rótulos do eixo Y
    paper_bgcolor='white',  # Fundo do papel do gráfico
    plot_bgcolor='white'   # Fundo da área do gráfico
)


# Mostrar o gráfico
fig.show()

### Evolução do ganho de medalhas pelo Brasil ao longo dos anos

In [54]:
import plotly.graph_objects as go

# Como o DataFrame contém tanto edições dos Jogos Olímpicos de Verão quanto de Inverno, vamos fazer dois gráficos separados a fim de evitar distorções e descobrir se existe alguma diferença no top 10 de países mais ganhadores em ambos os tipos de jogos.

# Começando pelos jogos de verão, primeiro criamos um novo DataFrame chamado "olimpiadas_verao" para filtrar as linhas do DataFrame "medalhas" que contenham a string "summer". Faremos isso para selecionar apenas os jogos de verão.

# Basicamente, aplicamos uma função lambda a cada linha (axis=1). Em seguida, a continuação x.astype(str).str.contains('summer', case=False).any() converte cada elemento da linha em string e verifica se esta contém a palavra "summer". Caso sim, a função retorna True e o DataFrame final contém apenas linhas em que "summer" está presente.

olimpiadas_verao = medalhas[medalhas.apply(lambda x: x.astype(str).str.contains('summer', case=False).any(), axis=1)]

# Em seguida, extraímos os anos das edições dos jogos olímpicos com .str.extract (considerando os primeiros 4 dígitos) e os transformamos em número inteiro.
olimpiadas_verao['year'] = olimpiadas_verao['edition'].str.extract(r'(\d{4})').astype(int)

# Repetimos o mesmo processo realizado no DataFrame "medalhas_grouped" para obter o número total de medalhas por país em cada edição das Olimpíadas de Verão.

olimpiadas_verao_grupo = olimpiadas_verao.groupby(['year', 'country'])['total'].sum().reset_index()

# Agora, criamos um novo DataFrame "brasil" para filtrar apenas as linhas que contêm os valores do Brasil.

brasil = olimpiadas_verao_grupo[olimpiadas_verao_grupo['country'].isin(['Brazil'])]

# Agrupando por país e somando o total de medalhas ao longo dos anos, teremos:
total_brasil = brasil.groupby('country')['total'].sum().reset_index()

# Finalmente (ufa!), vamos plotar o gráfico!

# Usando os dados do DataFrame 'brasil' para o gráfico, teremos:
data = [
    go.Scatter(
        x=brasil['year'],
        y=brasil['total'],
        mode='lines+markers+text',
        name='Brazil',
        text=brasil['total'],  # Adiciona os valores das medalhas como texto
        textposition='top center',  # Posição do texto
        textfont=dict(
            size=12,
            color='black'
        ),
        hovertext=brasil['total'],
        hoverinfo='text'
    )
]

# Por fim, definimos o layout do gráfico:
layout = go.Layout(
    title={
        'text': 'Evolução número de Medalhas do Brasil nas Olimpíadas',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='Ano',
    yaxis_title='Número de Medalhas',
    yaxis=dict(
        showticklabels=False,
        ),  # Remove os rótulos do eixo Y
    paper_bgcolor='white',
    plot_bgcolor='white'
)

# Finalmente, criamos a figura,
fig = go.Figure(data=data, layout=layout)

# E mostramos o gráfico!
fig.show()


<ipython-input-54-4971bc95ca8a>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

